In [ ]:
from collection import defaultdict

def max_sum():
    max_sum_dict = defaultdict(int)

Abc object with name: example and counter: 1
Abc2 object with name: example2 and counter: 2
example2Yash
None
True


In [15]:
from typing import List

class Solution:
    def productExceptSelf(self, nums: List[int]) -> List[int]:
        n = len(nums)
        answer = [1] * n
        
        print("Initial answer array:", answer)
        
        prefix_product = 1
        print("\n== Prefix Pass ==")
        for i in range(n):
            answer[i] = prefix_product
            print(f"answer[{i}] = {prefix_product}")
            prefix_product *= nums[i]
            print(f"Updated prefix_product *= nums[{i}] ({nums[i]}) => {prefix_product}")
        
        print("\nAnswer after prefix pass:", answer)
        
        suffix_product = 1
        print("\n== Suffix Pass ==")
        for i in range(n - 1, -1, -1):
            print(f"Before: answer[{i}] = {answer[i]}")
            answer[i] *= suffix_product
            print(f"Multiplying with suffix_product {suffix_product} => answer[{i}] = {answer[i]}")
            suffix_product *= nums[i]
            print(f"Updated suffix_product *= nums[{i}] ({nums[i]}) => {suffix_product}")
        
        print("\nFinal answer array:", answer)
        return answer

# Run the solution
sol = Solution()
nums = [30, 20, 40, 60]
result = sol.productExceptSelf(nums)
print("\nResult:", result)


Initial answer array: [1, 1, 1, 1]

== Prefix Pass ==
answer[0] = 1
Updated prefix_product *= nums[0] (30) => 30
answer[1] = 30
Updated prefix_product *= nums[1] (20) => 600
answer[2] = 600
Updated prefix_product *= nums[2] (40) => 24000
answer[3] = 24000
Updated prefix_product *= nums[3] (60) => 1440000

Answer after prefix pass: [1, 30, 600, 24000]

== Suffix Pass ==
Before: answer[3] = 24000
Multiplying with suffix_product 1 => answer[3] = 24000
Updated suffix_product *= nums[3] (60) => 60
Before: answer[2] = 600
Multiplying with suffix_product 60 => answer[2] = 36000
Updated suffix_product *= nums[2] (40) => 2400
Before: answer[1] = 30
Multiplying with suffix_product 2400 => answer[1] = 72000
Updated suffix_product *= nums[1] (20) => 48000
Before: answer[0] = 1
Multiplying with suffix_product 48000 => answer[0] = 48000
Updated suffix_product *= nums[0] (30) => 1440000

Final answer array: [48000, 72000, 36000, 24000]

Result: [48000, 72000, 36000, 24000]


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

########################################
# Synthetic Data Generation Functions  #
########################################

def generate_sine_wave(freq, duration=1.0, sample_rate=8000, amplitude=0.5):
    """
    Generates a sine wave with the specified parameters.
    """
    t = np.linspace(0, duration, int(sample_rate * duration), endpoint=False)
    return amplitude * np.sin(2 * np.pi * freq * t)

def generate_data(num_samples, sample_rate=8000, duration=1.0):
    """
    Generates synthetic dataset with cover signals.
    Here, we only generate the cover because the secret will be generated by the model.
    Cover: fixed frequency sine wave (e.g. 440 Hz).
    """
    covers = []
    for _ in range(num_samples):
        cover = generate_sine_wave(440, duration, sample_rate)
        covers.append(cover)
    return np.array(covers)

########################################
# Model Definitions                    #
########################################

class SecretGenerator(nn.Module):
    """
    Generates a secret signal from the cover audio.
    """
    def __init__(self):
        super(SecretGenerator, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=9, padding=4)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(16, 1, kernel_size=9, padding=4)
    
    def forward(self, cover):
        x = self.relu(self.conv1(cover))
        secret = self.conv2(x)
        return secret

class EmbeddingModel(nn.Module):
    """
    The embedding network takes the cover and secret audio (each a 1D signal)
    and outputs a perturbation that is added to the cover to yield the stego audio.
    """
    def __init__(self):
        super(EmbeddingModel, self).__init__()
        # Concatenate cover and secret: 2 channels
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=16, kernel_size=9, padding=4)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=1, kernel_size=9, padding=4)
    
    def forward(self, cover, secret):
        # Both inputs: (batch, 1, length)
        x = torch.cat([cover, secret], dim=1)  # shape: (batch, 2, length)
        x = self.relu(self.conv1(x))
        perturbation = self.conv2(x)
        stego = cover + perturbation
        return stego, perturbation

class DecoderModel(nn.Module):
    """
    The decoder network attempts to recover the secret signal from the stego audio.
    """
    def __init__(self):
        super(DecoderModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=9, padding=4)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=1, kernel_size=9, padding=4)
    
    def forward(self, stego):
        x = self.relu(self.conv1(stego))
        recovered = self.conv2(x)
        return recovered

class DetectionModel(nn.Module):
    """
    The detection network is a binary classifier (0: cover, 1: stego)
    based on a simple 1D CNN.
    """
    def __init__(self):
        super(DetectionModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=9, padding=4)
        self.relu = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=4)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=9, padding=4)
        self.pool2 = nn.MaxPool1d(kernel_size=4)
        # After two poolings, signal length becomes: 8000 / 4 / 4 = 500
        self.fc1 = nn.Linear(32 * 500, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x shape: (batch, 1, length)
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

########################################
# Training Functions                   #
########################################

def train_steganography(secret_generator, embedding_model, decoder_model, data_loader, num_epochs=10, lr=1e-3, lambda_cover=0.5, lambda_secret=1.0):
    """
    Trains the secret generator, embedding, and decoder networks jointly.
    The loss is a weighted sum of:
      - Cover loss: MSE between cover and stego audio.
      - Secret reconstruction loss: MSE between the generated secret and recovered secret.
    """
    criterion = nn.MSELoss()
    params = list(secret_generator.parameters()) + list(embedding_model.parameters()) + list(decoder_model.parameters())
    optimizer = optim.Adam(params, lr=lr)
    
    secret_generator.train()
    embedding_model.train()
    decoder_model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for (covers,) in data_loader:
            covers = covers.to(device)
            
            optimizer.zero_grad()
            # Generate secret from cover
            secret = secret_generator(covers)
            # Embed the generated secret into the cover
            stego, _ = embedding_model(covers, secret)
            # Decode the secret from the stego audio
            recovered = decoder_model(stego)
            
            loss_cover = criterion(stego, covers)
            loss_secret = criterion(recovered, secret)
            loss = lambda_cover * loss_cover + lambda_secret * loss_secret
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Steganography Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(data_loader):.6f}")

def train_detection_model(detection_model, secret_generator, embedding_model, covers, num_epochs=10, batch_size=16, lr=1e-3):
    """
    Trains the detection model to classify audio signals as cover (0) or stego (1).
    Uses the trained secret_generator and embedding_model to produce stego signals.
    """
    criterion = nn.BCELoss()
    optimizer = optim.Adam(detection_model.parameters(), lr=lr)
    detection_model.train()
    
    # Generate stego signals using the secret generator and embedding model
    with torch.no_grad():
        covers_tensor = torch.tensor(covers, dtype=torch.float32).unsqueeze(1).to(device)
        secret = secret_generator(covers_tensor)
        stego, _ = embedding_model(covers_tensor, secret)
        stego = stego.cpu().numpy().squeeze(1)  # Remove extra dimension
    
    # Create labels
    labels_cover = np.zeros(len(covers))
    labels_stego = np.ones(len(stego))
    
    # Combine cover and stego signals (both now have shape (num_samples, signal_length))
    signals = np.concatenate([covers, stego], axis=0)
    labels = np.concatenate([labels_cover, labels_stego], axis=0)
    
    # Create DataLoader for detection model
    signals_tensor = torch.tensor(signals, dtype=torch.float32).unsqueeze(1)
    labels_tensor = torch.tensor(labels, dtype=torch.float32)
    detection_dataset = TensorDataset(signals_tensor, labels_tensor)
    detection_loader = DataLoader(detection_dataset, batch_size=batch_size, shuffle=True)
    
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        correct = 0
        total = 0
        for signals, labels in detection_loader:
            signals = signals.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = detection_model(signals).view(-1)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
            predicted = (outputs > 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.float()).sum().item()
        accuracy = 100 * correct / total
        print(f"Detection Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(detection_loader):.6f}, Accuracy: {accuracy:.2f}%")


########################################
# Main Function                        #
########################################

def main():
    # Parameters
    num_samples = 1000       # Number of synthetic audio samples
    sample_rate = 8000       # Sampling rate in Hz
    duration = 1.0           # Duration in seconds
    batch_size = 16
    num_epochs = 10          # Number of epochs for training
    
    # Generate synthetic data (covers only)
    covers = generate_data(num_samples, sample_rate, duration)
    
    # Convert data to PyTorch tensors with shape (N, 1, signal_length)
    covers_tensor = torch.tensor(covers, dtype=torch.float32).unsqueeze(1)
    
    # Create DataLoader for steganography training
    stego_dataset = TensorDataset(covers_tensor)
    stego_loader = DataLoader(stego_dataset, batch_size=batch_size, shuffle=True)
    
    # Instantiate models
    secret_generator = SecretGenerator().to(device)
    embedding_model = EmbeddingModel().to(device)
    decoder_model = DecoderModel().to(device)
    detection_model = DetectionModel().to(device)
    
    print("Training the Adaptive Audio Steganography Model...")
    train_steganography(secret_generator, embedding_model, decoder_model, stego_loader, num_epochs=num_epochs, lr=1e-3)
    
    print("\nTraining the Detection Model...")
    train_detection_model(detection_model, secret_generator, embedding_model, covers, num_epochs=num_epochs, batch_size=batch_size, lr=1e-3)
    
    # Evaluate the detection model on a sample batch
    detection_model.eval()
    with torch.no_grad():
        covers_tensor = torch.tensor(covers, dtype=torch.float32).unsqueeze(1)
        secret = secret_generator(covers_tensor.to(device))
        stego, _ = embedding_model(covers_tensor.to(device), secret)
        # Create a small batch of cover and stego signals
        signals = torch.cat([covers_tensor.to(device), stego], dim=0)
        outputs = detection_model(signals).view(-1)
        predicted = (outputs > 0.5).float()
        print("\nSample Detection Results:")
        print("Predicted labels:", predicted.cpu().numpy())

if __name__ == "__main__":
    main()


Training the Adaptive Audio Steganography Model...
Steganography Epoch 1/10, Loss: 0.002349
Steganography Epoch 2/10, Loss: 0.000057
Steganography Epoch 3/10, Loss: 0.000029
Steganography Epoch 4/10, Loss: 0.000024
Steganography Epoch 5/10, Loss: 0.000019
Steganography Epoch 6/10, Loss: 0.000017
Steganography Epoch 7/10, Loss: 0.000014
Steganography Epoch 8/10, Loss: 0.000012
Steganography Epoch 9/10, Loss: 0.000011
Steganography Epoch 10/10, Loss: 0.000009

Training the Detection Model...
Detection Epoch 1/10, Loss: 0.694292, Accuracy: 50.40%
Detection Epoch 2/10, Loss: 0.693367, Accuracy: 50.00%
Detection Epoch 3/10, Loss: 0.693291, Accuracy: 50.00%
Detection Epoch 4/10, Loss: 0.693259, Accuracy: 50.00%
Detection Epoch 5/10, Loss: 0.693226, Accuracy: 50.00%
Detection Epoch 6/10, Loss: 0.693218, Accuracy: 50.00%
Detection Epoch 7/10, Loss: 0.693198, Accuracy: 50.00%
Detection Epoch 8/10, Loss: 0.693209, Accuracy: 50.00%
Detection Epoch 9/10, Loss: 0.693204, Accuracy: 50.00%
Detection 